In [1]:
import pandas as pd

df = pd.read_csv("../data/loans.csv")
df.head()



,Loan ID,Status,Client ID,Debtor Identifier,Debtor Identifier Type,Debtor Country,Trust ID,Amount,Created At,Accepted At,Refused At,Repaid At,Due Date,Insurance Status
0,ccba7b5961ac84c6bc09ba20b0497bd666ac10a7ecf123...,REPAID,2477304f4867e7ea86fd4414e0f845c0e4bd06516fe687...,17c277f8d264ccec868dc55add915dc93fd4ae4bd779cf...,siren,FR,e98a46aea01b6f55341744cdddbba3d6a88ab1e7d366eb...,13643.52,2025-01-24T09:52:41.912,2025-01-24T09:52:54.08,NaN,2025-03-06T05:05:38.362,2025-03-25T08:00:00,SETTLED
1,ea43f967166a7809d0e0f27fff33a3732fa600513d1b5f...,REPAID,a57af78c8caa2a9c6efbf8d9535f34441319c66ba56803...,494f60d81e3f8e6effbf093a1c2c6d0a99ca9d561bf624...,siren,FR,e98a46aea01b6f55341744cdddbba3d6a88ab1e7d366eb...,58.97,2025-08-27T07:51:48.154,2025-08-27T09:59:54.827,NaN,2025-09-30T10:30:19,01/10/2025,NaN
2,1d39f1f6c61cef090d31c552b0d3e511280ba64c2fb69a...,REPAID,dd3a54bd37885757d4c4b3dbd85c5fdcffa351fe2d1680...,8ab1efec5e005f628c0e5793276ddaf1e3693cf880a0a9...,cif,ES,e98a46aea01b6f55341744cdddbba3d6a88ab1e7d366eb...,6654.03,2025-02-24T09:13:01.171,2025-02-24T16:51:01.077,NaN,2025-06-03T18:32:50,2025-05-20T09:12:33.344,REFUSED
3,f03b65936792e9d35e66db0572aa43cdc5f2d33d75fcd0...,REPAID,dd3a54bd37885757d4c4b3dbd85c5fdcffa351fe2d1680...,8563f72a004fcba1d5ae23410ccf82a6d8bcc85fb6ccfd...,kvk,NL,c4ed1a68f3ad7b3c85c4400e688e4dd3dcfe4da53d171d...,4452.80,2025-10-07T06:19:38.206,2025-10-07T06:19:47.087,NaN,2025-10-29T15:34:56,2025-11-01T06:19:09.316,SETTLED
4,70290cf7ced390115c4443cab5f519fcf8ac52a11dafbd...,REPAID,dd3a54bd37885757d4c4b3dbd85c5fdcffa351fe2d1680...,17c277f8d264ccec868dc55add915dc93fd4ae4bd779cf...,siret,FR,e98a46aea01b6f55341744cdddbba3d6a88ab1e7d366eb...,966.08,2025-07-29T12:21:45.349,2025-07-29T12:21:54.306,NaN,2025-09-05T14:43:14,2025-09-22T12:21:41.397,SETTLED


In [2]:
df.columns = (
    df.columns
      .str.strip()
      .str.lower()
      .str.replace(" ", "_")
)

df.columns.tolist()


['loan_id',
 'status',
 'client_id',
 'debtor_identifier',
 'debtor_identifier_type',
 'debtor_country',
 'trust_id',
 'amount',
 'created_at',
 'accepted_at',
 'refused_at',
 'repaid_at',
 'due_date',
 'insurance_status']

In [3]:
df["accepted_at"] = pd.to_datetime(df["accepted_at"], errors="coerce")
df["amount"] = pd.to_numeric(df["amount"], errors="coerce")

monthly_production = (
    df.loc[df["accepted_at"].notna()]                
      .assign(month=lambda d: d["accepted_at"].dt.to_period("M").dt.to_timestamp())  
      .groupby("month", as_index=False)["amount"]
      .sum()
      .rename(columns={"amount": "monthly_production"})
      .sort_values("month")
      .reset_index(drop=True)
)

monthly_production
monthly_production_display = monthly_production.copy()
monthly_production_display["monthly_production"] = monthly_production_display["monthly_production"].map(
    lambda x: f"{x:,.0f}" if pd.notna(x) else ""
)

monthly_production_display


,month,monthly_production
0,2025-01-01,"17,365,329"
1,2025-02-01,"25,049,322"
2,2025-03-01,"26,840,395"
3,2025-04-01,"27,682,889"
4,2025-05-01,"28,552,878"
5,2025-06-01,"29,736,636"
6,2025-07-01,"29,929,763"
7,2025-08-01,"24,574,723"
8,2025-09-01,"25,868,885"
9,2025-10-01,"31,054,042"


In [4]:
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd

min_date = monthly_production["month"].min().date()
max_date = monthly_production["month"].max().date()

from_date = widgets.DatePicker(
    description="From:",
    value=min_date
)

to_date = widgets.DatePicker(
    description="To:",
    value=max_date
)

chart_type = widgets.Dropdown(
    options=["Line Chart", "Bar Chart"],
    value="Line Chart",
    description="Chart:",
    layout=widgets.Layout(width="300px")
)

output = widgets.Output()

def render(_=None):
    with output:
        clear_output(wait=True)

        if from_date.value is None or to_date.value is None:
            print("Please select both From and To dates.")
            return

        start = pd.Timestamp(from_date.value)
        end = pd.Timestamp(to_date.value)

        if start > end:
            print("From date must be earlier than To date.")
            return

        filtered = monthly_production.loc[
            (monthly_production["month"] >= start) &
            (monthly_production["month"] <= end)
        ]

        if filtered.empty:
            print("No data in the selected timeframe.")
            return

        title = "Monthly Production Since Inception (Financed Amount)"

        if chart_type.value == "Line Chart":
            fig = px.line(
                filtered,
                x="month",
                y="monthly_production",
                markers=True,
                title=title
            )
        else:
            fig = px.bar(
                filtered,
                x="month",
                y="monthly_production",
                title=title
            )

        fig.update_layout(
            xaxis_title="Month",
            yaxis_title="Monthly Production",
            yaxis_tickformat="~s",
            xaxis=dict(
                tickmode="linear",
                dtick="M1",
                tickformat="%b %Y"
            )
        )

        fig.update_traces(
            hovertemplate=(
                "<b>%{x|%b %Y}</b><br>"
                "Monthly Production: %{y:,.0f}<extra></extra>"
            )
        )

        fig.show()

from_date.observe(render, names="value")
to_date.observe(render, names="value")
chart_type.observe(render, names="value")

display(widgets.HBox([from_date, to_date, chart_type]), output)
render()

Output()